In [0]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer


#df = spark.read.format('parquet').table('training_dataset_numeric')
df = spark.read.format('parquet').table('training_dataset_numeric')
df.write.format("delta").mode("overwrite").save("/delta/events")

df = spark.read.format('delta').load('/delta/events') 
df = df.select('bathrooms_na','price','minimum_nights','property_type','beds')
df = df.na.drop()

In [0]:
df = df.filter(df["property_type"].isin(['Private room in condominium (condo)', 'Entire residential home', 'Entire rental unit']))
                        
display(df.
       groupby('property_type').count())

property_type,count
Private room in condominium (condo),99
Entire residential home,536
Entire rental unit,1260


In [0]:
# without hyperparameters tuning

# Transform our Train & Test in delta format
(Train, Test) = df.randomSplit([.8,.2], seed=42)

# categorial vs numerical
categoricalCols = [field for (field, dataType) in Train.dtypes if dataType == "string"]
indexOutputCols = [x + "Index" for x in categoricalCols]
stringIndexer = StringIndexer(inputCols = categoricalCols, outputCols = indexOutputCols, handleInvalid='skip')

numericalCols = [field for (field, dataType) in Train.dtypes if ((dataType == 'double') & (field != 'price'))]

assemblerInputs = indexOutputCols + numericalCols
vecAssembler = VectorAssembler(inputCols = assemblerInputs, outputCol = 'features')


rf = RandomForestRegressor(labelCol="price", maxBins=40)
stages = [stringIndexer, vecAssembler, rf]
pipeline = Pipeline(stages = stages)

predDF = pipeline.fit(Train).transform(Test)


In [0]:
print(rf.explainParams())

bootstrap: Whether bootstrap samples are used when building trees. (default: True)
cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). default = 'auto' (default: auto)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: variance (default: variance)
labelCol: label column name. (default: label, current: price)
leafCol: Leaf indices column name. Predicted leaf index of each instance in each tree by preorder. (default: )
maxBins: Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature. (default: 32, current: 40)
maxDepth: Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 5)
maxMemoryInMB: Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size. (default: 256)
minInfoGain: Minimum information gain for a split to be considered at a tree node. (default: 0.0)
minInstancesPerNode: Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1. (default: 1)
minWeightFractionPerNode: Minimum fraction of the weighted sample count that each child must have after split. If a split causes the fraction of the total weight in the left or right child to be less than minWeightFractionPerNode, the split will be discarded as invalid. Should be in interval [0.0, 0.5). (default: 0.0)
numTrees: Number of trees to train (>= 1). (default: 20)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: 2502083311556356884)
subsamplingRate: Fraction of the training data used for learning each decision tree, in range (0, 1]. (default: 1.0)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol = 'price', predictionCol = 'prediction')
rmse = evaluator.evaluate(predDF)
r2 = evaluator.setMetricName('r2').evaluate(predDF)

print(rmse)
print(r2)

184.6839422735982
0.32698469283833875

In [0]:
# with hyperparameters tuning => directly connected with mlflow in background, allows an interative comparaison
from pyspark.ml.tuning import ParamGridBuilder

paramGrid = (ParamGridBuilder()
            .addGrid(rf.maxDepth, [2,5])
            .addGrid(rf.numTrees, [5,10])
            .build())

paramGrid

Out[17]: [{Param(parent='RandomForestRegressor_34d5ed4d355d', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2,
 Param(parent='RandomForestRegressor_34d5ed4d355d', name='numTrees', doc='Number of trees to train (>= 1).'): 5},
 {Param(parent='RandomForestRegressor_34d5ed4d355d', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2,
 Param(parent='RandomForestRegressor_34d5ed4d355d', name='numTrees', doc='Number of trees to train (>= 1).'): 10},
 {Param(parent='RandomForestRegressor_34d5ed4d355d', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='RandomForestRegressor_34d5ed4d355d', name='numTrees', doc='Number of trees to train (>= 1).'): 5},
 {Param(parent='RandomForestRegressor_34d5ed4d355d', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='RandomForestRegressor_34d5ed4d355d', name='numTrees', doc='Number of trees to train (>= 1).'): 10}]

In [0]:
# 3K cross validation
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator

evaluator = RegressionEvaluator(labelCol = 'price', predictionCol = 'prediction')
cv = CrossValidator(estimator = pipeline, evaluator = evaluator, estimatorParamMaps = paramGrid, numFolds = 3, seed = 42)

In [0]:
cvModel = cv.fit(Train)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [0]:
# problem : manually search space so we can miss the optimum one and does have an expense
# Hyperopt solves this by doing the hyperparameter based on a range of value, could be run with Serial or Parallel
# 3 options : 
# Random search
# TPE (tree of parzen Estimators)
# Adaptive TPE - freeze parameter 1 after result then search for parameter 2

In [0]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import mlflow

def objective_function(params):
  max_depth = params['max_depth']
  num_trees = params['num_trees']
  print(max_depth)
  
  grid = (ParamGridBuilder()
            .addGrid(rf.maxDepth, ['max_depth'])
            .addGrid(rf.numTrees, ['num_trees'])
            .build())
  
  cv = CrossValidator(estimator = pipeline, evaluator = evaluator, estimatorParamMaps = paramGrid, numFolds = 3, seed = 42)
  cvModel = cv.fit(Train)
  
  rmse = cvModel.avgMetrics[0]
  
  return ('loss', rmse, "status", STATUS_OK)

In [0]:
from hyperopt import hp

search_space = {
  "max_depth" : hp.choice('max_depth', np.arange(1, 10,dtype=int)),
  "num_trees" : hp.choice('num_trees', np.arange(10, 20,dtype=int))
}

In [0]:
from hyperopt import fmin, tpe, STATUS_OK, Trials
import numpy as np 

with mlflow.start_run():
  num_evals = 4
  trials = Trials()
  
  best_hyperparam = fmin(fn = objective_function,
                         space = search_space,
                         algo = tpe.suggest,
                         max_evals = num_evals, 
                         trials = trials,
                         rstate = np.random.RandomState(42)
                        )
  
  best_max_depth = best_hyperparam['max_depth']
  best_num_trees = best_hyperparam['num_trees']
  
  # run RF with the best param
  rf.setMaxDepth(best_max_depth)
  rf.setNumTrees(best_num_trees)
  
  pipelineModel = pipeline.fit(trainDF)
  
  predDF = pipelineModel.transform(Test)
  rmse = regressionEvaluator.evaluate(predDF)
  
  mlflow.log_param('max_depth', best_max_depth)
  mlflow.log_param('num_trees', best_max_depth)
  mlflow.log_metric('rmse', rmse)